Arbol elemental con libreria  rpart

Debe tener instaladas las librerias  data.table  ,  rpart   y rpart.plot

In [1]:
#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'rpart.plot'"


Aqui se debe poner la carpeta de SU computadora local

In [2]:
setwd("C:/Users/vyago/Desktop/Maestría Ciencias de Datos/07-DMEYF")


cargo el dataset

In [3]:
dataset  <- fread("./datasets/competencia1_2022.csv")

defino donde entreno y donde aplico el modelo

In [4]:
# Creamos una clase binaria
dataset[, clase_binaria := ifelse(
                            clase_ternaria == "BAJA+2",
                                "evento",
                                "noevento"
                            )]

dataset[,clase_ternaria:=NULL]

In [5]:
dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar
dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

genero el modelo,  aqui se construye el arbol

In [17]:
modelo  <- rpart(formula=    "clase_binaria ~ .", #quiero predecir clase_ternaria a partir de el resto de las variables
                 data =      dtrain,  #los datos donde voy a entrenar
                 xval=       0,
                 cp=        -0.3, #esto significa no limitar la complejidad de los splits
                 minsplit= 300,   #minima cantidad de registros para que se haga el split
                 minbucket=  1,   #tamaño minimo de una hoja
                 maxdepth=  17 )  #PRUEBO  una profundidad de 10

Ahora grafico el arbol, va a ser muy frondoso porque la profundidad es de 10

Ahora aplico al modelo  a los datos muevos  y genero la salida para kaggle

aplico el modelo a los datos nuevos

In [18]:
prediccion  <- predict( modelo, dapply , type = "prob") #aplico el modelo

prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
cada columna es el vector de probabilidades 

agrego a dapply una columna nueva que es la probabilidad de BAJA+2

In [23]:
dapply[ , prob_baja2 := prediccion[, "evento"] ]

solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/40

In [24]:
dapply[ , Predicted  := as.numeric(prob_baja2 > 1/40) ]

Finalmente genero el archivo para Kaggle

creo la carpeta donde va el experimento

In [11]:
#dir.create( "./exp/KA2002" ) 

grabo el dataset

In [25]:
dapply[,.N,by=Predicted]

Predicted,N
<dbl>,<int>
0,155525
1,7375


In [44]:
dtrain.corr()

ERROR: Error in dtrain.corr(): no se pudo encontrar la función "dtrain.corr"


In [26]:
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/KA2002/KA2002_002.csv", 
        sep= "," )